- 模型 SD2-1-base：https://huggingface.co/stabilityai/stable-diffusion-2-1-base
- 模型 SD2-1：https://huggingface.co/stabilityai/stable-diffusion-2-1
- 模型 SD1.5：https://huggingface.co/runwayml/stable-diffusion-v1-5
- 模型 SD1.4：https://huggingface.co/CompVis/stable-diffusion-v1-4

In [ ]:
#@title GPU检测
!nvidia-smi

In [ ]:
#@title 安装依赖库
!pip install diffusers transformers scipy ftfy accelerate

#选择模型并加载

In [8]:
model_id = "stabilityai/stable-diffusion-2-1-base" #@param ["runwayml/stable-diffusion-v1-5", "CompVis/stable-diffusion-v1-4", "stabilityai/stable-diffusion-2-1-base", "stabilityai/stable-diffusion-2-1"] 

In [ ]:
from diffusers import StableDiffusionPipeline
# 避免内存超限，使用半精度（可以删除 torch_dtype 参数，使用全精度，但需要非常高内存）
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

pipe = pipe.to("cuda")

#生成

In [ ]:
#@title 简单生成
import torch

# 定义生成器，0 表示使用随机种子
generator = torch.Generator("cuda").manual_seed(0)

prompt = "a city, Cyberpunk" #@param

# 生成并显示图片
image = pipe(prompt, generator=generator).images[0] 

# Colab 可以直接输出显示
image

In [ ]:
#@title 指定更多参数生成

#@markdown 如果某个随机数生成图片很复合预期，可以指定随机数一直生成相同图片，得到确定性输出

seed = 0 #@param 

#@markdown 推理步数会影响出图效果，步数越多，细节越多效果越好。一般不超过50，20-30之间就效果不错
steps = 25 #@param

#@markdown 指导因子指数，影响图片与文本的关联度，数值越大多样性越小，关联性越强。一般在 7-9 即可
scale = 7 #@param

#@markdown 指定宽高，默认 512*512，确保宽高都是8的倍数，不要设置太大
height = 512 #@param
width = 512 #@param


generator = torch.Generator("cuda").manual_seed(seed)

image = pipe(
      prompt, 
      generator=generator, 
      num_inference_steps=steps, 
      guidance_scale=scale,
      width=width,
      height=height
    ).images[0]

image

In [ ]:
#@title 同时生成多张

from PIL import Image

# 先使用 PIL 模块定义图片展示区域的助手函数
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

#@markdown 指定生成数量
num_images = 3 #@param

images = pipe(
        [prompt] * num_images,
        generator=generator, 
        num_inference_steps=steps, 
        guidance_scale=scale,
        width=width,
        height=height
      ).images

# 以一行三列形式展示
grid = image_grid(images, rows=1, cols=3)
grid
